# Crack processing

The crack propagation and state characterization is done using the class `DICCrack` which represents its geometry in form of a spline.
Cracks are contained in the `DICCrackList` and can be accessed using the index operator with the crack label. Currently, the crack label is an enumeration running from left to right. 

In [ ]:
%matplotlib widget
from bmcs_shear.dic_crack import sz_tests_series_2023 as ts
import ibvpy.api as ib
import matplotlib.pylab as plt
from matplotlib import cm
from bmcs_shear.dic_crack import\
    DICGrid, DICInpUnstructuredPoints, DICStateFields, DICCrackList, \
    DICAlignedGrid
import numpy as np
np.seterr(divide ='ignore', invalid='ignore');
import collections
collections.Iterable = collections.abc.Iterable

In [ ]:
dcl = ts.new_dcl(ts.B1_TV2)

In [ ]:
dc = dcl.cracks[0]
self = dc.sp
self.dic_grid.t = 0.1
eps_unc_t_Lab = self.eps_unc_t_Lab
X_unc_t_La = self.X_unc_t_La

In [ ]:
fig, ax = plt.subplots(1,1)
ax.plot(X_unc_t_La[:,1], eps_unc_t_Lab[:,0,0])
ax.set_xlabel(r'$y$/mm')
ax.set_ylabel(r'$\varepsilon$/-')
ax.set_title('strain profile over the height')

In [ ]:
import numpy as np
from scipy import signal

def autocorr(x):
    result = np.correlate(x, x, mode='full')
    return result[result.size // 2:]

def autocorr_length(x):
    ac = autocorr(x - np.mean(x))
    ac = ac / np.max(ac)    # Normalize
    return np.where(ac < 0)[0][0]

X_L = X_unc_t_La[:,1]
eps_L = eps_unc_t_Lab[:,0,0]

d_X = X_L[1] - X_L[0]
corr_length = autocorr_length(eps_L) * d_X
print(f'Estimated autocorrelation length: {corr_length:.1f} mm')

In [ ]:
cmm = self.bd.matrix_
mdm = self.smeared_matmod
mdm.trait_set(E=cmm.E_c, nu=0.2)
n_K, _, _ = eps_unc_t_Lab.shape
Eps = {
    name: np.zeros((n_K,) + shape)
    for name, shape in mdm.state_var_shapes.items()
}
sig_t_unc_Lab, _ = mdm.get_corr_pred(eps_unc_t_Lab, 1, **Eps)
sig_t_unc_Lab

In [ ]:
dc = dcl.cracks[3]
sp = dc.sp
fig, (ax_N, ax_V, ax_M) = plt.subplots(1,3, figsize=(13,3))
sp.plot_ST(ax_N, ax_V, ax_M)

In [ ]:
%matplotlib widget
fig, ax = plt.subplots(1,1)
#dcl.plot_cracking_hist2(ax)
#dc.plot_omega_t_Ni(ax)
ax.plot(dc.omega_t_N, dc.x_1_Ka[:,1], color='blue')
ax.fill_between(dc.omega_t_N, dc.x_1_Ka[:,1], color='blue', alpha=0.4)
#ax.axis('equal')
#ax.axis('off')

In [ ]:
dc.M_N

In [ ]:
N = np.arange(len(dc.M_N))

In [ ]:
xx_MN, yy_MN, cd_field_irn_MN = dcl.dsf.crack_detection_ipl_field
y_N = yy_MN[0,:]

In [ ]:
fig, ax = plt.subplots(1,1)
ax.plot(cd_field_irn_MN[dc.M_N, N], y_N)

In [ ]:
dc.x_t_tip_a[1], dc.x_1_tip_a[1]

In [ ]:
dc.x_1_Ka[dc.K_tip_T[dc.T_t]]

In [ ]:
dc.K_tip_T

In [ ]:
xx_MN, yy_MN, cd_field_irn_MN = dcl.dsf.crack_detection_ipl_field
cd_field_irn_MN[np.where(cd_field_irn_MN > 0)]